In [ ]:
import huggingface_hub
hf_token = '' # put your User Access Tokens here
# ابتدا login کنید
huggingface_hub.login(token=hf_token)

# سپس وضعیت ورود را بررسی کنید
!hf auth whoami

/home/ai/Desktop/Fine-Tune/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


user:  AM-Nateghi


In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorWithPadding
from peft import prepare_model_for_kbit_training
import torch

cptk = "google/gemma-3-1b-pt"
from transformers import BitsAndBytesConfig
tokenizer = AutoTokenizer.from_pretrained(cptk)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
model = AutoModelForCausalLM.from_pretrained(
    cptk,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,
)
model = prepare_model_for_kbit_training(model)

model.config.use_cache = False # for Gradient checkpointing
model.gradient_checkpointing_enable()

In [18]:
from peft import LoraConfig, get_peft_model, TaskType

lora_conf = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias='none',
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_conf)

In [23]:
from datasets import load_dataset

dataset = load_dataset(
    "json", data_files={"train": "assets/qa_train.json", "test": "assets/qa_test.json"}
)


def formatter(batch):
    tokened = tokenizer(batch["input"], truncation=True, max_length=1024, padding="max_length")
    tokened["labels"] = tokenizer(
        batch["output"], truncation=True, max_length=1024, padding="max_length"
    )["input_ids"]

    return tokened


tokenized_dataset = dataset.map(formatter, batched=True)

Map: 100%|██████████| 48/48 [00:00<00:00, 1995.70 examples/s]


In [24]:
torch.cuda.empty_cache()

In [25]:
# اگر مدل فاقد pad_token است، آن را به eos-tokenنسبت دهید:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, return_tensors="pt")

In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback

callbacks = [EarlyStoppingCallback(early_stopping_patience=8)]

training_args = TrainingArguments(
    output_dir="./gemma_qlora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=5,
    eval_steps=50,
    save_steps=50,
    save_total_limit=2,
    metric_for_best_model="loss",
    save_strategy='steps',
    eval_strategy="steps",
    gradient_checkpointing=True,
    load_best_model_at_end=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    callbacks=callbacks,
    data_collator=data_collator,
)
trainer.train()

Truncating eval dataset: 100%|██████████| 48/48 [00:00<00:00, 14538.32 examples/s]


Step,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.00 GiB. GPU 0 has a total capacity of 11.59 GiB of which 4.19 GiB is free. Including non-PyTorch memory, this process has 6.96 GiB memory in use. Of the allocated memory 6.15 GiB is allocated by PyTorch, and 595.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)